In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy
from scipy import optimize
from IPython.display import display, Math, Latex
from scipy import stats
from tqdm import tqdm

# Parent folder
import pathlib
from pathlib import Path  
pf = pathlib.Path().resolve() # Points to parent folder containing notebook and data, eg: 'C:/Users/Carla/Dropbox/Uni/10. Semester/Dynamic Programming/Term paper'

# Step 1: Constrained Least Squares

$$
\min _{\left\{m_{i}\right\}_{i=1}^{n} \in \mathbb{R}^{n}} \sum_{i=1}^{n}\left(m_{i}-y_{i}\right)^{2}
$$

\begin{equation}
0 \leq \frac{m_{i+1}-m_{i}}{k_{i+1}-k_{i}} \leq e^{-r \tau} \text { for } i=1, \ldots, n-1 \text { (monotonicity) }
\end{equation}

(monotonicity constraint for puts shown, inequality reversed for calls)

\begin{equation}
\frac{m_{i+1}-m_{i}}{k_{i+1}-k_{i}} \leq \frac{m_{i+2}-m_{i+1}}{k_{i+2}-k_{i+1}} \text { for } i=1, \ldots, n-2 \text { (convexity) }
\end{equation}


Conducting a constrained least squares optimization on each cross section of option prices (over strike prices $k$, for fixed time $t$ and maturity $\tau$)

The problem of constrained least squares regression consists in finding the closest values $m_{i}$, in the sense of least squares, to a set of $n$ observations $y_{1},y_{2},...,y_{n}$ 
satisfying a set of constraints.



## Define objective and constraints:

In [ ]:
def objective(x):
    """ Returns the squared difference between a value vector x and actual prices y, note that prices must be a global defined variable y """
    return sum((x-y)**2)

def monotonicity_constraint(x):
    """ Checks for monotonicity in prices, note that strikes k must be a global defined variable, and it also uses 'cp_flag' to tell if put or call! """
    dx = x[1:] - x[:len(x)-1]
    dk = k[1:] - k[:len(k)-1]
    
    if cp_flag == 'P':
        return disc[:len(disc)-1] - (dx/dk)  
    if cp_flag == 'C':
        return  disc[:len(disc)-1] + (dx/dk)  
    
def convexity_constraint(x):
    """ Checks for convexity in prices """
    dx = x[1:] - x[:len(x)-1]
    dk = k[1:] - k[:len(k)-1]
    ddx = dx[1:]
    ddk = dk[1:]
    return ((ddx / ddk) - (dx[:len(x)-2] / dk[:len(k)-2]))

## Run the regression for calls and puts, for each date and tau

In [6]:
# full period
start_dates = ['1996-01-01','1997-01-01','1998-01-01','1999-01-01','2000-01-01','2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01',
          '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']
end_dates = ['1996-12-31','1997-12-31','1998-12-31','1999-12-31','2000-12-31','2001-12-31','2002-12-31','2003-12-31','2004-12-31','2005-12-31','2006-12-31','2007-12-31','2008-12-31','2009-12-31','2010-12-31','2011-12-31','2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
          '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']

names = ['SPX','NDX','DJX', 'RUT', 'TSLA', 'AMZN', 'GOOGL']

In [ ]:
for name in names:
    for start, end in zip(start_dates, end_dates):

        # Read processed data file from optiondataprep (by year)
        filepath = Path(f'{pf}/data/output/allopt_01Jan{start[0:4]}to31Dec{end[0:4]}_extract_{name}.csv') # Set name 
        df_main = pd.read_csv (filepath)  

        df_main = df_main.sort_values(by=['date','exdate', 'tau', 'strike'])

        #Ensure date is filtered correctly
        df_main = df_main[df_main['date'].between(start, end)]

        df_main['p_adj'] = np.nan # Create nan column for p_adj to fill with regression predictions
        nrows = len(df_main)

        if nrows > 1: # Continue with estimation

            # All combinations of flag, date, tau that we need to calculate the regression for
            loop_vars = df_main.groupby(['cp_flag','date','tau']).size().reset_index()
            length = len(loop_vars)

            # Estimation loop
            for i, (cp_flag, date, tau) in enumerate(tqdm(zip(loop_vars['cp_flag'], loop_vars['date'], loop_vars['tau']), total=length, position=0, leave=True, desc=start[0:4])): # For loop with progress bar

                ### Setup data and variables for regression

                # Select fixed date (date)
                df = df_main[df_main.date == date]

                # Select put or call only (cp_flag )
                df = df[df['cp_flag'] == cp_flag]

                # Select fixed maturity (tau)
                df = df[df['tau'] == int(tau)]

                if len(df) < 3: # Should not be neccessary if passing full data set as this filter is already applied earlier, but is a must for testing otherwise regression will fail
                    continue

                else:

                    ### Regression

                    # Setup regression variables
                    y = df['price'].values # Actual price
                    k = df['strike'].values # Strikes
                    r = df['dy_ma'].values[0] # Risk free rate
                    tau = df['tau'].values[0] # Tau
                    tau_years = df['tau_years'].values[0] # Tau
                    #disc = np.exp(-df['dy_ma'].values[0]*df["tau_years"].values)
                    disc = np.exp(-df['tr'].values[0]*df["tau_years"].values)

                    # Setup bounds and constraints
                    bounds = scipy.optimize.Bounds(lb= 0, ub=np.inf, keep_feasible=False) # lower bound = 0, upper bound = max price * 10
                    c1 = {'type': 'ineq', 'fun': monotonicity_constraint} 
                    c2 = {'type': 'ineq', 'fun': convexity_constraint}

                    # Initial values:
                    x0 = y+0.001

                    # Call optimizer
                    result = optimize.minimize(objective,x0, method='SLSQP', bounds=bounds, constraints=[c1, c2], options={'disp':False, 'ftol':1e-7, 'maxiter':1000})

                    # Collect results along with date, flag, tau and strike in one data frame
                    frames = [df['date'], df['cp_flag'], df['tau'], df['strike'], pd.DataFrame(result.x, columns = ['p_adj'], index=df.index)] 
                    df_results = pd.concat(frames, axis=1, ignore_index=True)
                    df_results.columns = ['date', 'cp_flag', 'tau', 'strike', 'p_adj']

                    # Update main data frame with results (p_adj)
                    df_main.update(df_results)

            # Add column with predicted price vs actual price
            df_main['p_adj_error_pct']= (df_main['p_adj']-df_main['price'])/df_main['price']*100

            # Save the results
            filepath = Path(f'{pf}/data/output/cls_regression_nrows_{start[0:4]}-{end[0:4]}_{name}.csv') # Set name 
            df_main.to_csv(filepath, index=False)
        else:
            print(f'No data for {name}, {start[0:4]}')
            continue
    print(name, ' - done', len(df))


In [ ]:
for name in names:
    df_all = pd.DataFrame()
    # Merge data from each year to one file
    for start, end in zip(start_dates, end_dates):
        filepath = Path(f'{pf}/data/output/cls_regression_nrows_{start[0:4]}-{end[0:4]}_{name}.csv') # Set name 
        
        try:
            df_results = pd.read_csv(filepath)
        except FileNotFoundError:
            print(f'No data for {name}, {start_dates[0:4]}, {end_dates[0:4]}')
            continue
        
       #df_results = pd.read_csv(filepath)
        df_all = pd.concat((df_all, df_results))

    # Save the results
    filepath = Path(f'{pf}/data/output/cls_regression_nrows_{start_dates[0][0:4]}-{end_dates[-1][0:4]}_{name}.csv') # Set name 
    df_all.to_csv(filepath, index=False)
    
    # Checking for error in actual price vs predicted price
    print(name, 'error pct>50: ',len(df_all[df_all.p_adj_error_pct>50]), 'total: ',len(df_all))

# Step 2: Local Polynomial Fitting

We want to obtain the CDF of the state-price distribution $Q^*(s)$

$Q^*(s)$ is connected to the first order derivative of the pricing functions (for puts and calls respectively)

\begin{aligned}
&\hat{Q}_p^*(s):=e^{r \tau} \hat{m}_p^{(1)}(s) \\
&\hat{Q}_c^*(s):=e^{r \tau} \hat{m}_c^{(1)}(s)+1
\end{aligned}

To obtain first order derivatives (dropping subscript here as its the same) ${m}^{(1)}(s)$ we need to fit a polynomial on the strike prices and the adjusted option prices (p_adj)

nonparametric local polynomial fit (Fan and Yao 2005)




**Process :**
1. Local nonparametric polynomial fit of order 2  - Estimating the beta matrix $\begin{equation}
\hat{\beta}:=\left(X^{\prime} W X\right)^{-1} X^{\prime} W y
\end{equation}$, etc
2. Return the first derivative $\hat{m}_o^{(1)}(s)$, standard error, and optimal bandwitdh etc
3. Estimate the CDF state-price distribution $\hat{Q}_o^*(s):=e^{r \tau} \hat{m}_o^{(1)}(s)+1\{o=c\} \quad \text { for } o=c, p$

## Local nonparametric polyfit function

In [4]:
# Utility function for newey-west covariance
def phi(x):
    return np.exp((-x**2)/2) / np.sqrt(2*np.pi)

# Utility function for newey-west covariance
def nw_cov(q, m):
    mu = np.mean(q)
    n = len(q)
    gam = np.zeros(int(m))
    cov = 0

    for j in range(1, int(m)+1):
        A = q[0:n-j]-mu
        B = q[0+j:n]-mu
        gam[j-1] = sum(A * B) /n
        cov += (1-j/(m+1))*gam[j-1]
    return(cov)

# Utility function to make cdf and se valid
def make_valid(qcdf_out, qcdf_se_out):
    # Empty arrays
    qcdf = np.zeros(len(qcdf_out))
    qcdf_se_pt = np.zeros(len(qcdf_out))
    
    # Make Q and SE valid
    for i in range(len(qcdf)):
        qcdf[i] =  min(1, max(0, qcdf_out[i]))
        if i > 1 and qcdf[i-1] == 1:
            qcdf[i] = 1
        if qcdf[i] == 0 or qcdf[i] == 1:
            qcdf_se_pt[i] = 0
        else:
            qcdf_se_pt[i] = qcdf_se_out[i]
            
    return(qcdf, qcdf_se_pt)
    
def lpoly(kin, vk, vc, p, opth, h0):
    """ 
    input variables: 
    # kin   current strike price (scalar)
    # vk    vector of strikes (column vector, nk x 1)
    # vc    vector of option prices (column vector, nk x 1)
    # p     power p 
    # opth  switch for bandwidth 
    #           0 = use h0
    #           1 = optimal local bandwidth
    #           2 = optimal global bandwidth
    # h0    user-specified bandwidth

    # output variables:
    # beta    local polynomial estimate and first derivative (p+1)x1
    # beta_se standard error
    # h     optimal bandwidth
    # sign  singular matrix """
    
    
    # Setting initital hcon
    if p == 1:
        hcon = 0.776
    elif p == 2:
        hcon = 0.884
    elif p == 3:
        hcon = 1.006

    # Number of strikes
    nk=len(vk)
    
    # Setting
    hnumsd = 3
    sign = 0

    ### Optimal bandwidth
    if opth == 1 or opth == 2:
        x = np.ones((nk,1))
        d = 0 

        while d<p+3:
            d = d + 1
            x = np.concatenate((x, vk[:, None]**d), axis=1)
            
        A = (np.transpose(x) @ x)
        B =  np.transpose(x) @ vc
        alpha = np.linalg.solve(A, B)

        res = vc - (x@alpha)
        ssr = sum(res**2)

        w0 = lambda v=None: (v > np.mean(vk) - 1.5 *  np.std(vk)) * (v < np.mean(vk) + 1.5*np.std(vk))
        mp1 = lambda v=None: math.factorial(p+1)*alpha[p+1] + 1/2*math.factorial(p+2)*alpha[p+2]*v +1/6*math.factorial(p+3)*alpha[p+3]*v**2 # Evaluate 3rd derivative
        wmp1= sum(mp1(vk)**2 * w0(vk))

        h = h0 

    ### Compute local polynomial estimate
    XX=np.ones((nk,1))
    d=0
    nu=1

    while d < p:
        d=d + 1
        d_fact = math.factorial(d)

        # Create matrices
        XX=np.column_stack((XX, (vk - kin) ** d)) # Matrix: 1, (X-x0)^1, (X-x0)^2, ... , (X-x0)^p  
        nu=np.vstack((nu, d_fact))

    # Calculate beta as the solution to the linear matrix equation # X^T W X * beta = X^T W y.
    W=np.diag(phi((vk - kin) / h) / h) # W is a (p+ 1)x(p+ 1) diagonal matrix whose i'th element is Kh(ki-s) (Kh(x) =h^-1*K(x/h), where K is the kernel and h is the bandwidth
    A = (np.transpose(XX) @ W) @ XX
    B = (np.transpose(XX) @ W) @ vc
    beta = np.linalg.solve(A , B) # the derivative m′(x) is estimated by the local slope ratherthan the derivative of the estimated regression function - b[1] is the derivative


    ### Compute standard error of local polynomial estimate
    ps = 0 # pow for smoothing s2 hat
    #hs = h # h for smoothing s2 hat 
    hs = h0 # h for smoothing s2 hat 

    # Computing Sigma (S)
    A = np.linalg.solve(nu**2*(np.transpose(XX) @ W) @ XX, ((np.transpose(XX) @ W) @ W) @ XX)
    B = (np.transpose(XX) @ W) @ XX
    S = np.linalg.solve(B.conj().T, A.conj().T).conj().T # Answer 2 -  https://stackoverflow.com/questions/1007442/mrdivide-function-in-matlab-what-is-it-doing-and-how-can-i-do-it-in-python
    
    # Squared residuals
    resid2 = (vc - beta[0])**2  
    resid2[np.isnan(resid2)]=0
    resid2[np.isinf(resid2)]=0

    XX=np.ones((nk,1))
    d=0
    nu=1
    while d < ps:
        d=d + 1
        kk = (vk - kin)**d    
        XX = np.concatenate((XX, kk[:, None]), axis=1)
        d_fac = scipy.special.factorial(d)
        nu = np.vstack((nu, d_fac))
    
    W = np.diag(phi((vk-kin)/hs)/hs)  
    A = ((np.transpose(XX) @ W) @ XX)
    B = (np.transpose(XX) @ W) @ resid2
    s2hat = np.transpose(nu) * np.linalg.solve(A, B) # (ps+1) x 1
    beta_var = s2hat*S
    beta_se = np.real(np.sqrt(np.real(np.diag(beta_var))))

    # Optimal local bandwidth
    if opth == 1:        
        fhat = np.mean(phi((vk-kin)/hs)/hs)
        h = hcon*(np.mean(resid2)/mp1(kin)**2/fhat/nk)**(1/(2*p+3))        

    return(beta, beta_se, h, sign)

# Step 3: Point and Interval Estimation of Bubbles

We define a function that cmputes cdf and se based on local polynomial estimation, the function will evaluate over a grid of strikes from the lowest
     strike to the highest strike on the selected date, where ds (delta S) is the step size in the grid
   
For each date, tau for puts and calls respectively we compute the cdf estimates and asset value:
1. Using Lpoly - estimate the CDF state-price distribution $\hat{Q}_o^*(s):=e^{r \tau} \hat{m}_o^{(1)}(s)+1\{o=c\} \quad \text { for } o=c, p$
2. Normalize CDF by equation (12): $Q^{\dagger}(s):=\frac{Q^*(s)-Q^*(\ell)}{Q^*(u)-Q^*(\ell)} \quad \text { for } s \in[\ell, u]$
3. Estimate the fundamental asset value by (17): $\hat{\mu}^{\dagger}:=e^{-r \tau} \ell+e^{-r \tau} \sum_{i=1}^{n_s}\left[1-\hat{Q}^{\dagger}\left(s_i\right)\right] \Delta s_i$ 
...
4. Compute the standard error of estimates

We then combine the estimates and compute bubbles:
1. Compute the weighted estimate of the fundamental asset value $\hat{\mu}^{\dagger}=\omega \hat{\mu}_c^{\dagger}+(1-\omega) \hat{\mu}_p^{\dagger}(\omega=[0,1])$ where the **weights are the proportion of put/call contracts** , and the standard error of the estimate
2. Compute the bias $\hat{B}$
3. Compute the bubble (uncorrected) by $\hat{\Pi}^S=S+\hat{B}-\hat{\mu}^{\dagger}$

## Define function for computing cdf estimates

In [5]:
def compute_cdf_estimates(df_main, cp_flag, date, tau, ds):
    """ Computes cdf and se based on local polynomial estimation, the function will evaluate over a grid of strikes from the lowest
     strike to the highest strike on the selected date, where ds (delta S) is the step size in the grid
     
    input variables: 
    df        df containing optiondata with CLS adjusted prices
    cp_flag   switch for put/call
    date      fixed date for run estimation
    tau       fixed tau for estimation
    
    output variables:
    qcdf          cumulative distribution function
    qcdf_se_out   standard errror of cdf (before correction) """
    
    
     # Select fixed date (date)
    df = df_main[df_main['date'] == date]
    
    # Drop observations where we dont have estimate for p_adj
    df = df[df['p_adj'].notna()]

    # Select fixed maturity (tau)
    df = df[df['tau'] == int(tau)]
    
    # Select put or call only (cp_flag )
    df = df[df['cp_flag'] == cp_flag]
        
    # Evaluate over range of strikes
    ds = ds # step size - delta s
    lb = min(df['strike']) # lower bound of strike range
    ub = max(df['strike']) # Upper bound of strike range

    k_range = np.arange(lb, ub+ds, ds)
    
    #print(f'lb{lb}, ub{ub}, k_range {len(k_range)}')

    # Discount factor
    #dis = np.exp(-df['dy_ma'].values[0]*df["tau_years"].values[0])
    dis = np.exp(-df['tr'].values[0]*df["tau_years"].values[0])
    

    # -- # "Calculate qcdf_put by equation (14) " # -- #
    # Local polynomial fit -> Evaluate qcdf with strikes - (first derivative from lpoly) 
    vk = np.array(df['strike'])
    vc = np.array(df['p_adj'])

    # Local regression parameters
    p = 2
    hnumsd = 3 #number of mean dk set for h0 (initial bandwidth) 
    opth = 1

    # Initial bandwitdh
    dk = df['strike'].diff()
    hx0 = np.mean(dk)*hnumsd

    qcdf_out, qcdf_se_out = [], []
    for i, kin in enumerate(k_range):
        
        # Run estimation
        b, b_se, h, sign = lpoly(kin, vk, vc, p, opth, hx0) # to get h opt
        b, b_se, h, sign = lpoly(kin, vk, vc, p, opth, h) # with updated h
        

        if cp_flag == 'P':
            qcdf_out.append(b[1]*dis**(-1))
            qcdf_se_out.append(b_se[1]*dis**(-1))

        elif cp_flag == 'C':
            qcdf_out.append(1+b[1]*dis**(-1))
            qcdf_se_out.append(b_se[1]*dis**(-1))


    # make Q and se valid
    qcdf, qcdf_se_pt = make_valid(qcdf_out, qcdf_se_out)

    # -- # "Normalize cdf by equation (12)" # -- #
    qcdf_norm = (qcdf - min(qcdf))/(max(qcdf)-min(qcdf))

    # -- # "Calculate estimate of fundamental value by equation (17)" # -- #
    qcdf_mu = (lb + np.nansum((1-qcdf_norm)*ds))*dis
    qcdf_mu = (np.nansum(1 - qcdf_norm) * ds + vk[0])*dis

    # Compute standard error of estimate
    mp = np.ceil(len(k_range)**0.25)
    wp = ((np.ones(len(k_range)) * ds)* dis) / (max(qcdf)-min(qcdf))
    
    if cp_flag == 'P':
        wp[0] = (dis*ds*sum(qcdf) - (df['price'].values[-1] - df['price'].values[0])) / (max(qcdf)-min(qcdf))**2 + dis*ds / (max(qcdf)-min(qcdf))
        wp[-1] = -(dis*ds*sum(qcdf) - (df['price'].values[-1] - df['price'].values[0])) / (max(qcdf)-min(qcdf))**2 + dis*ds / (max(qcdf)-min(qcdf))
    if cp_flag == 'C':
        wp[0] = (dis*ds*sum(qcdf) - (df['price'].values[-1] - df['price'].values[0]+dis*(vk[-1]-vk[0]) )) / (max(qcdf)-min(qcdf))**2 + dis*ds / (max(qcdf)-min(qcdf))
        wp[-1] = -(dis*ds*sum(qcdf) - (df['price'].values[-1] - df['price'].values[0]+dis*(vk[-1]-vk[0]))) / (max(qcdf)-min(qcdf))**2 + dis*ds / (max(qcdf)-min(qcdf))


    qcdf_norm_se_pt = qcdf_se_pt * wp 
    qcdf_pt = qcdf * wp
    qcdf_se=np.sqrt(np.nansum(qcdf_norm_se_pt ** 2) + 2*nw_cov(qcdf_pt, mp))

    # Additional returns
    n = len(df) # number put or calls in this subset of data
    vol = sum(df['volume']) # volume

    #print(f' The estimated fundamental value by {cp_flag, date, tau} is {qcdf_mu:.2f} with standard error {qcdf_se:.2f}')

    return(qcdf, qcdf_mu, qcdf_se, n, dis, vol, df, sign) 


## Run estimation for each date, tau (intersect)

In [8]:
# Weighting scheme
weights = [False, True, 'Naive'] #Volume weighted = True / False, if false then jarrow weights
volume_weighted = 'True' 

In [ ]:
for name in names:
    for start, end in zip(start_dates, end_dates):
        # Read data (from step 1 - CLS)
        filepath = Path(f'{pf}/data/output/cls_regression_nrows_{start[0:4]}-{end[0:4]}_{name}.csv') # Set name 
        try:
            df_main = pd.read_csv(filepath)
            
        except FileNotFoundError:
            print(f'No data for {name}, {start[0:4]}')
            continue
            

        if len(df_main) > 1: # Continue with estimation

            # All combinations of date, tau that we need to run the estimations for
            loop_vars = df_main.groupby(['date','tau']).size().reset_index()
            length = len(loop_vars)
            variables = ['date', 'sout', 'tau', 'qcdf_mu', 'qcdfp_mu', 'qcdfc_mu','sbub_qcdf', 'sbub_qcdfp', 'sbub_qcdfc', 'otmc', 'call1', 'sbub_qcdfp_se', 'sbub_qcdfc_se', 'sbub_qcdf_se', 'qcdfp_bias',
                   'qcdfc_bias', 'qcdf_bias', 'qcdf_A_lb', 'qcdf_A_ub', 'qcdf_Ap_lb', 'qcdf_Ap_ub', 'qcdf_Ac_lb', 'qcdf_Ac_ub' ,
                   'qcdf_B1', 'qcdf_B21', 'qcdf_B22', 'qcdf_B23','qcdf_B3', 'Bcbub_lb', 'Bcbub_ub', 'scene', 'qcdfp_lb', 'qcdfp_ub',
                   'qcdfc_lb', 'qcdfc_ub', 'lp', 'up', 'lc', 'uc', 'nkc', 'nkp', 'sumvolc', 'sumvolp', 'B1p', 'B1c', 'B2p', 'B2c', 'sign']

            # For saving results
            results = np.zeros((length, len(variables)-1)) #nrows, ncolums (number of variables)
            index, dates = [], []

            for i, (date, tau) in enumerate(tqdm(zip(loop_vars['date'], loop_vars['tau']), total=length, position=0, leave=True, desc=start[0:4])): # For loop with progress bar
                # Set date and tau (loop)
                date, tau = date, tau

                ### Setup data and variables for estimation

                # Select fixed date (date)
                df = df_main[df_main['date'] == date]

                # Select fixed maturity (tau)
                df = df[df['tau'] == int(tau)]
                
                # For k-range - fineness of evaluation grid
                ds = np.mean(np.diff(df['strike']))
                
                # Count number of observations for each cpflag, tau , if either is less than or equal to 3 then skip this date:
                ptaucount = df[df.cp_flag == 'P'].shape[0]
                ctaucount = df[df.cp_flag == 'C'].shape[0]
                if ptaucount <= 3 or ctaucount <= 3:
                    index.append(i), dates.append('not enough observations')
                    continue

                else:
                    sign = 0

                    # Compute individual estimates using calls and puts
                    qcdfp, qcdfp_mu, qcdfp_se, nputs, dis, volp, df_p, sign = compute_cdf_estimates(df, 'P', date, tau, ds) # put estimates
                    qcdfc, qcdfc_mu, qcdfc_se, ncalls, dis, volc, df_c, sign = compute_cdf_estimates(df, 'C', date, tau, ds) # call estimates

                    # Weights used - currently number of puts/calls for give date, tau
                    if volume_weighted == True:
                        nputs = volp
                        ncalls = volc
                        n = nputs+ncalls
                    if volume_weighted == 'Naive':
                        nputs = 1
                        ncalls = 1
                        n = nputs+ncalls

                    # Combined estimate
                    n = nputs + ncalls # 
                    qcdf_mu = nputs/n*qcdfp_mu + ncalls/n*qcdfc_mu        
                    qcdf_se = np.sqrt((nputs/n*qcdfp_se)**2 + (ncalls/n*qcdfc_se)**2)

                    # Set up strike and price arrays for bias calculations
                    if df_p.size == 0:
                        pk = np.empty(0)
                        put = np.empty(0)
                    else:
                        pk = df_p['strike'].values
                        put = df_p['price'].values

                    if df_c.size == 0:
                        ck = np.empty(0)
                        call = np.empty(0)
                    else:
                        ck = df_c['strike'].values
                        call = df_c['price'].values

                    # Find individual biases
                    try:
                        lc_p = np.argwhere(pk>=ck[0])[0]
                    except IndexError:
                        lc_p = np.argwhere(pk<=ck[0])[-1]
                    try:
                        up_c = np.argwhere(ck<=pk[-1])[-1]
                    except IndexError:
                        up_c = np.argwhere(ck>=pk[-1])[0]
                    try:
                        lp_c = np.argwhere(ck>=pk[0])[0]
                    except IndexError:
                        lp_c = np.argwhere(ck<=pk[0])[-1]
                    try:
                        uc_p = np.argwhere(pk<=ck[-1])[-1]
                    except IndexError:
                        uc_p = np.argwhere(pk>=ck[-1])[0]

                    B1p = dis * min(qcdfp) * (pk[-1] - pk[0]) / (max(qcdfp)-min(qcdfp))
                    B1c = dis * min(qcdfc) * (ck[-1] - ck[0]) / (max(qcdfc)-min(qcdfc))

                    B2p = (1/(max(qcdfp)-min(qcdfp))-1)*(put[-1] - put[0])
                    B2c = (1/(max(qcdfc)-min(qcdfc))-1)*(dis*(ck[-1]-ck[0]) + call[-1] - call[0])

                    A_lb = -put[0]
                    A_ub = call[-1]
                    Ap_lb = -put[0]
                    Ap_ub = call[up_c]
                    Ac_lb = -put[lc_p]
                    Ac_ub = call[-1]

                    # Compute bias for combined put and call estimate
                    B1 = (nputs*B1p + ncalls*B1c)/n
                    B21 = (nputs*B2p + ncalls*B2c)/n
                    B3 = nputs/n*dis*(ck[-1]-pk[-1])

                    if pk[0] <= ck[0] and pk[-1] <= ck[-1] and ck[0] <= pk[-1]:
                        B22 = nputs/n*(dis*(ck[-1]-pk[-1]) + call[up_c] - call[up_c])
                        B23 = ncalls/n*(put[lc_p] - put[0])
                        scene = 1

                    elif ck[0] <= pk[0] and ck[-1] <= pk[-1] and pk[0] <= ck[-1]:
                        B22 = -nputs/n*(put[-1] - put[uc_p])
                        B23 = -ncalls/n*(dis*(pk[0]-ck[0]) + call[lp_c] - call[0])
                        scene = 2

                    elif pk[0] <= ck[0] and ck[-1] <= pk[-1]:
                        B22 = ncalls/n*(put[lc_p] - put[0])
                        B23 = -nputs/n*(put[-1] - put[uc_p])
                        scene = 3

                    elif ck[0] <= pk[0] and pk[-1] <= ck[-1]:    
                        B22 = -ncalls/n*(dis*(pk[0]-ck[0]) + call[lp_c] - call[0])
                        B23 = nputs/n*(dis*(ck[-1]-pk[-1]) + call[-1] - call[up_c])
                        scene = 4

                    elif pk[-1] < ck[0]:
                        B22 = ncalls/n*(put[-1] - put[0])
                        B23 = nputs/n*(dis*(ck[-1]-ck[0]) + call[-1] - call[0])
                        A_ub = call[-1] + dis
                        scene = 5
                    else:
                        scene = 6


                    ### Set up output variables
                    qcdfp_bias = B1p - B2p                
                    qcdfc_bias = B1c - B2c
                    qcdf_bias = B1 - B21 + B22 + B23 - B3
                    qcdf_A_lb = A_lb
                    qcdf_A_ub = A_ub
                    qcdf_Ap_lb = Ap_lb
                    qcdf_Ap_ub = Ap_ub
                    qcdf_Ac_lb = Ac_lb
                    qcdf_Ac_ub = Ac_ub
                    qcdf_B1 = B1
                    qcdf_B21 = B21
                    qcdf_B22 = B22
                    qcdf_B23 = B23
                    qcdf_B3 = B3

                    # Compute bubble
                    sout = df['snp'].values[0] # Underlying price
                    sbub_qcdfp = sout - qcdfp_mu  # 100% put information
                    sbub_qcdfc = sout - qcdfc_mu # 100% call information
                    sbub_qcdf = sout - qcdf_mu  # weighted bubble estimate
                    
                    # Standard errors
                    sbub_qcdfp_se = qcdfp_se 
                    sbub_qcdfc_se = qcdfc_se 
                    sbub_qcdf_se = qcdf_se
                    #
                    qcdfp_lb = min(qcdfp)
                    qcdfp_ub = max(qcdfp)
                    qcdfc_lb = min(qcdfc)
                    qcdfc_ub = max(qcdfc)

                    lp = pk[0]
                    up = pk[-1]
                    lc = ck[0]
                    uc = ck[-1]
                    nkp = nputs
                    nkc = ncalls
                    sumvolc = volc
                    sumvolp = volp
                    otmc = call[-1]
                    call1 = call[0]
                    Bcbub_lb = -(1/(max(qcdfc)-min(qcdfc))-1)*call[0]
                    Bcbub_ub = -(1/(max(qcdfc)-min(qcdfc))-1)*call[-1]

                     # save results
                    index.append(i), dates.append(date)
                    for j, var in enumerate(variables[1:]):
                        results[i, j] = globals()[var]
                        
                    #print(f'df_main {start[0:4]}, {lp}, {up}')

            # Format results
            df_dates = pd.DataFrame(dates)
            df_results = pd.DataFrame(results)
            df_results = pd.concat([df_dates, df_results], axis=1)
            df_results.columns = variables
            df_results=df_results[df_results.date != 'not enough observations']
            df_results['date'] =  pd.to_datetime(df_results['date'], format='%Y-%m-%d')

            # Save estimation results
            filepath = Path(f'{pf}/data/output/bub_interval_estimates_nrows_{start[0:4]}-{end[0:4]}_VW{volume_weighted}_{name}.csv') # Set name 
            df_results.to_csv(filepath, index=True)
        else:
            print(f'No data for {name}, {start[0:4]}')
            continue

In [ ]:
for name in names:
    df_all = pd.DataFrame()
    # Merge data from each year to one file
    for start, end in zip(start_dates, end_dates):
        filepath = Path(f'{pf}/data/output/bub_interval_estimates_nrows_{start[0:4]}-{end[0:4]}_VW{volume_weighted}_{name}.csv') # Set name 
        try:
            df_results = pd.read_csv(filepath)
            #print(filepath)
        except FileNotFoundError:
            #print(f'No data for {name}, {start[0:4]}')
            continue
        #df_results = pd.read_csv(filepath)
        df_all = pd.concat((df_all, df_results), ignore_index=True)

    # Save the results
    filepath = Path(f'{pf}/data/output/bub_interval_estimates_final_{start_dates[0][0:4]}-{end_dates[-1][0:4]}_VW{volume_weighted}_{name}_all.csv') # Volume_weighted = False
    df_all.to_csv(filepath, index=False)
    #df_all.to_excel('testx.xlsx')
    print(f'done {name}, {len(df_all)}')

# Bub plots

In [ ]:
def bub(name, start_dates, end_dates, volume_weighted, semax):
    ma = 63
    i = names.index(name)

    # Read data
    filepath= Path(f'{pf}/data/output/bub_interval_estimates_final_{start_dates[0][0:4]}-{end_dates[-1][0:4]}_VW{volume_weighted}_{name}_all.csv')
    print(filepath)

    df_main = pd.read_csv(filepath)

    df_main['date'] =  pd.to_datetime(df_main['date'], format='%Y-%m-%d')
    df_main = df_main.sort_values(['date', 'tau'], ascending = [True, True])

    # Add maturity groups and bias corrected bubble estimate
    df_main['tau_years'] = df_main['tau']/365 # tau in years
    df_main['maturity_group'] = np.where(df_main['tau_years']<0.25, "low", np.where(df_main['tau_years']>0.5, "high", "med")) # group tau_years by, 0-0.25 = low, 0.25-0.50 = med, tau_years > 0.5 = high
    #df_main['maturity_group'] = np.where(df_main['tau_years']<8/365, "vlow",  np.where((df_main['tau_years']<0.25)&(df_main['tau_years']<0.25), "low", np.where(df_main['tau_years']>0.5, "high", "med"))) #


    # Smooth bubble estimates
    print(len(df_main))

    meancp = np.mean(df_main.sbub_qcdf)
    meanp = np.mean(df_main.sbub_qcdfp)
    meanc = np.mean(df_main.sbub_qcdfc)
    print(len(df_main))

    # Impute nonsensible values - replace with previous value if se > 1000
    df_main['sbub_qcdf_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdf_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdf_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdf_se"].ffill(inplace=True) # Fill NAN with previous valid obs

    df_main['sbub_qcdfp_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdfp_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdfp_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdfp_se"].ffill(inplace=True) # Fill NAN with previous valid obs

    df_main['sbub_qcdfc_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdfc_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdfc_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdfc_se"].ffill(inplace=True) # Fill NAN with previous valid obs
    
    df_main=df_main[df_main['qcdfc_bias']<1000]
    df_main=df_main[df_main['qcdfp_bias']<1000]
    df_main=df_main[df_main['qcdfc_bias']<1000]
    df_main=df_main[df_main['qcdfc_bias']>-1000]
    df_main=df_main[df_main['qcdfp_bias']>-1000]
    df_main=df_main[df_main['qcdfc_bias']>-1000]

    print(len(df_main))

    # Setting up loop and variables
    tau_grps = ['low', 'med', 'high', 'all']


    period = df_main.date.unique()
    nperiods = len(df_main.date.unique())
    ntaugrps = len(tau_grps)
    nem = 3 #number of estimation methods
    mc = np.ceil(ma**0.25);

    # for collecting results
    bub_gp = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_mu = np.zeros((nperiods, ntaugrps, nem)) ### Den her giver ikke mening, bare en tom en som de sætter ind i NW_COV, giver 0???
    sbubcdf_se = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_mu = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_lb = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_ub = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bias = np.zeros((nperiods, ntaugrps, nem))

    # Estimates by day and group
    for t, date in enumerate(period):
        for j, grp in enumerate(tau_grps):


            # Subset date
            df = df_main[df_main['date'] == date] # date

            # Select group
            if grp in ['low', 'med', 'high']: # if group is low, med or high, then select group otherwise include all
                # Subset grp
                df = df[df['maturity_group'] == grp] # group


            # Bias corrected estimates
            bub_gp[t, j, 0] = np.nanmean(df['sbub_qcdfp'] + df['qcdfp_bias']);
            bub_gp[t, j, 1] = np.nanmean(df['sbub_qcdfc'] + df['qcdfc_bias']);
            bub_gp[t, j, 2] = np.nanmean(df['sbub_qcdf'] + df['qcdf_bias']);
            sbubcdf_bc_mu[t, j, 0] = bub_gp[t, j, 0] 
            sbubcdf_bc_mu[t, j, 1] = bub_gp[t, j, 1] 
            sbubcdf_bc_mu[t, j, 2] = bub_gp[t, j, 2] 

            # Standard error 
            ngp = len(df)
            sbubcdf_se[t, j, 0] = np.sqrt(np.nansum(df['sbub_qcdfp_se']**2))/ngp;  
            sbubcdf_se[t, j, 1] = np.sqrt(np.nansum(df['sbub_qcdfc_se']**2))/ngp;  
            sbubcdf_se[t, j, 2] = np.sqrt(np.nansum(df['sbub_qcdf_se']**2))/ngp;  

            # Bounds of A - used for confidence interval
            sbubcdf_bc_lb[t, j, 0] = bub_gp[t, j, 0] - np.nanmean(df['qcdf_Ap_ub']);
            sbubcdf_bc_ub[t, j, 0] = bub_gp[t, j, 0] - np.nanmean(df['qcdf_Ap_lb']);
            sbubcdf_bc_lb[t, j, 1] = bub_gp[t, j, 1] - np.nanmean(df['qcdf_Ac_ub']);
            sbubcdf_bc_ub[t, j, 1] = bub_gp[t, j, 1] - np.nanmean(df['qcdf_Ac_lb']);
            sbubcdf_bc_lb[t, j, 2] = bub_gp[t, j, 2] - np.nanmean(df['qcdf_A_ub']);
            sbubcdf_bc_ub[t, j, 2] = bub_gp[t, j, 2] - np.nanmean(df['qcdf_A_lb']);

            #Bias
            sbubcdf_bias[t, j, 0] = np.nanmean(df['qcdfp_bias']);
            sbubcdf_bias[t, j, 1] = np.nanmean(df['qcdfc_bias']);
            sbubcdf_bias[t, j, 2] = np.nanmean(df['qcdf_bias']); 



    # For collecting results
    rsbubcdf_mu = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_se = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_mu = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_lb = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_ub = np.zeros((nperiods, ntaugrps, nem))

    # Moving averages:
    for t in np.arange(ma, nperiods):
        vt = np.arange(t-ma+1, t) # window of observations
        for j, grp in enumerate(tau_grps):
            for i in np.arange(0, nem): # estimation methods
                #print(t, vt, j)

                # Bubble moving average estimate
                rsbubcdf_bc_mu[t,j,i] = np.nanmean(sbubcdf_bc_mu[vt,j,i]);

                # Newey-West Standard error
                rsbubcdf_mu[t,j,i] = np.nanmean(sbubcdf_mu[vt,j,i])
                rsbubcdf_se[t,j,i] = np.sqrt((np.nansum(sbubcdf_se[vt,j,i]**2) + 2*nw_cov(sbubcdf_mu[vt,j,i],mc)))/ma

                # Confidence interval
                rsbubcdf_bc_lb[t,j,i] = np.nanmean(sbubcdf_bc_lb[vt,j,i]) - 1.96*rsbubcdf_se[t,j,i]
                rsbubcdf_bc_ub[t,j,i] = np.nanmean(sbubcdf_bc_ub[vt,j,i]) + 1.96*rsbubcdf_se[t,j,i]

    #print(f'mean bub est= {np.mean(df_main.sbub_qcdf)}, max bub est= {np.max(df_main.sbub_qcdf)}, min bub est= {np.min(df_main.sbub_qcdf)}')
    #print(f'mean se est= {np.mean(df_main.sbub_qcdf_se)}, max se est= {np.max(df_main.sbub_qcdf_se)}, min se est= {np.min(df_main.sbub_qcdf_se)}')
    #print(f'periods {nperiods}')
    
    return(rsbubcdf_bc_mu, rsbubcdf_bc_lb, rsbubcdf_bc_ub, period, bub_gp)

## Stocks

In [ ]:
names = ['TSLA', 'AMZN', 'GOOGL']
tsla = bub(names[0], start_dates, end_dates, volume_weighted, 1000)
amzn = bub(names[1], start_dates, end_dates, volume_weighted, 1000)
googl = bub(names[2], start_dates, end_dates, volume_weighted, 500)
frames = [tsla, amzn, googl]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)

colors = ['blue', 'orange', 'green', 'red']
start = start_dates[14]
end = end_dates[-1]
print(start, end)

frame = frames[2]

i = 2
j = 0

print(i,j)


# plot moving average for bubble estimate
ax.plot(tsla[3][:], tsla[0][:,j,i], label = f'TSLA', color=colors[0])
ax.plot(amzn[3][:], amzn[0][:,j,i], label = f'AMZN', color=colors[1])
ax.plot(googl[3][:], googl[0][:,j,i], label = f'GOOGL', color=colors[2])

# plot confidence interval
ax.fill_between(tsla[3][:], tsla[1][:,j,i], tsla[2][:,j,i], color=colors[0], alpha=.25)
ax.fill_between(amzn[3][:], amzn[1][:,j,i], amzn[2][:,j,i], color=colors[1], alpha=.25)
ax.fill_between(googl[3][:], googl[1][:,j,i], googl[2][:,j,i], color=colors[2], alpha=.25)

# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Bubble estimate')
ax.set_title(r'$\hat{\Pi}_{cp}(\tau)$', loc='center', fontsize='medium')
ax.legend(loc='upper left')
#ax.set_ylim(-60,200)

filepath = Path(f'{pf}/data/figures/bubbles_stocks') # Set name
plt.savefig(filepath) #bbox_inches='tight'
    
#ax.set_ylim(0, 100)
#ax.set_ylim(-60, 50)

## Indices

In [ ]:
names = ['SPX', 'NDX', 'RUT']
spx = bub(names[0], start_dates, end_dates, volume_weighted, 1000)
ndx = bub(names[1], start_dates, end_dates, volume_weighted, 1000)
rut = bub(names[2], start_dates, end_dates, volume_weighted, 1000)
frames = [spx, ndx, rut]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)
colors = ['blue', 'red', 'orange', 'green']
start = start_dates[0]
end = end_dates[-1]
print(start, end)

frame = frames[2]

i = 2
j = 0

print(i,j)


# plot moving average for bubble estimate
ax.plot(spx[3][:], spx[0][:,j,i], label = f'S&P 500', color=colors[0])
ax.plot(ndx[3][:], ndx[0][:,j,i]/5, label = f'Nasdaq', color=colors[1])
ax.plot(rut[3][:], rut[0][:,j,i], label = f'Russel 2000', color=colors[2])

# plot confidence interval
ax.fill_between(spx[3][:], spx[1][:,j,i], spx[2][:,j,i], color=colors[0], alpha=.25)
ax.fill_between(ndx[3][:], ndx[1][:,j,i]/5, ndx[2][:,j,i]/5, color=colors[1], alpha=.25)
ax.fill_between(rut[3][:], rut[1][:,j,i], rut[2][:,j,i], color=colors[2], alpha=.25)


# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Bubble estimate')
ax.set_title(r'$\hat{\Pi}_{cp}(\tau)$', loc='center', fontsize='medium')
ax.legend(loc='upper left')
#ax.set_ylim(-60,200)

filepath = Path(f'{pf}/data/figures/bubbles_indices') # Set name
plt.savefig(filepath) #bbox_inches='tight'
    
#ax.set_ylim(0, 100)
ax.set_ylim(-60, 200)